In [ ]:
library(dplyr)
library(data.table)
library(tidyr)
library(ggplot2)
library(patchwork)
library(svglite)
library(fgsea)

In [ ]:
contacts_file = snakemake@input$contacts
#read contacts
dfcnt = fread(cmd = paste("sed s/?/NA/g ",contacts_file))
colnames(dfcnt) <- paste("Cluster",1:ncol(dfcnt),sep="_")
dfcnt$Position = 1:nrow(dfcnt)

dfcnt_tmp <- dfcnt %>%
    pivot_longer(cols = starts_with("Cluster"),names_to = "Cluster",values_to = "Contact")

dfcnt_tmp <- dfcnt_tmp %>%
    group_by(Position) %>%
    filter(!is.na(Contact)) %>%
    summarise(Cluster_all=min(Contact,na.rm = T)) 

dfcnt <- dfcnt %>%
    left_join(dfcnt_tmp,by="Position") %>%
    select(Position,starts_with("Cluster")) %>%
    pivot_longer(cols = starts_with("Cluster"),names_to = "Cluster",values_to = "Contact") %>%
    mutate(ID = sub("Cluster_","",Cluster))

dfcntshort <- dfcnt %>%
    filter(Contact <= 4.5)

In [ ]:
options(repr.plot.width=9, repr.plot.height=7, jupyter.plot_mimetypes = "image/svg+xml")
contact_review <- ggplot(data=dfcnt, aes(x=Position,y=Contact)) + geom_point(aes(color=ID),size=0.5) + facet_wrap(~ID) + theme(text = element_text(size = 12), legend.position = "none") + ylab("Contact, Å")
contact_reviewshort <- ggplot(data=dfcntshort, aes(x=Position,y=Contact)) + geom_point(aes(color=ID),size=0.5) + facet_wrap(~ID) + theme(text = element_text(size = 12), legend.position = "none") + ylab("Contact, Å")

contact_reviewall <- contact_review /contact_reviewshort
contact_reviewall <- contact_reviewall + plot_annotation(tag_levels = 'a')
ggsave(snakemake@output[[1]], contact_reviewall, width = 9, height = 7)
contact_reviewall

In [ ]:
dfcnt4out <- dfcnt %>%
    select(-ID) %>%
    mutate(Contact = if_else(Contact <= 4.5,1,0)) %>%
    pivot_wider(names_from = Cluster, values_from = Contact)
dfcnt4out[dfcnt4out==0] <- NA
dfcnt4out[is.na(dfcnt4out)] <- 0
fwrite(x = dfcnt4out, file=snakemake@output[[2]])